# Writing an Algorithm that Uses DMAC Trading

## Data Loading

In [14]:
# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [15]:
# Read the aapl.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
aapl_df = pd.read_csv(
    Path("../Resources/aapl.csv"),
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True
)

In [18]:
# Review the DataFrame
aapl_df.head()

,close,volume,open,high,low
date,,,,,
2014-09-22,101.06,52421660,101.80,102.14,100.58
2014-09-23,102.64,63255860,100.60,102.94,100.54
2014-09-24,101.75,59974260,102.16,102.85,101.20
2014-09-25,97.87,99689300,100.51,100.71,97.72
2014-09-26,100.75,62276770,98.53,100.75,98.40


In [19]:
# Filter the date index and close columns
signals_df = aapl_df.loc[:, ["close"]]

# Review the DataFrame
signals_df

,close
date,
2014-09-22,101.06
2014-09-23,102.64
2014-09-24,101.75
2014-09-25,97.87
2014-09-26,100.75
...,...
2019-09-13,218.75
2019-09-16,219.90
2019-09-17,220.70


In [21]:
# Use hvplot to visualize the data
signals_df.hvplot()

:Curve   [date]   (close)

## SMA for the Short and Long Windows Calculation

In [22]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

In [23]:
# Generate the short and long window simple moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["close"].rolling(window=long_window).mean()
# Review the DataFrame
signals_df

,close,SMA50,SMA100
date,,,
2014-09-22,101.06,NaN,NaN
2014-09-23,102.64,NaN,NaN
2014-09-24,101.75,NaN,NaN
2014-09-25,97.87,NaN,NaN
2014-09-26,100.75,NaN,NaN
...,...,...,...
2019-09-13,218.75,207.0573,200.50975
2019-09-16,219.90,207.3707,200.63715
2019-09-17,220.70,207.7843,200.79135


## Trading Signals Identification

In [29]:
# Create a column to hold the trading signal
signals_df["Signal"] = 0.0

In [37]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
signals_df

,close,SMA50,SMA100,Signal
date,,,,
2014-09-22,101.06,NaN,NaN,0.0
2014-09-23,102.64,NaN,NaN,0.0
2014-09-24,101.75,NaN,NaN,0.0
2014-09-25,97.87,NaN,NaN,0.0
2014-09-26,100.75,NaN,NaN,0.0
...,...,...,...,...
2019-09-13,218.75,207.0573,200.50975,1.0
2019-09-16,219.90,207.3707,200.63715,1.0
2019-09-17,220.70,207.7843,200.79135,1.0


## Finding the Crossover Points

In [38]:
# Slice the DataFrame to confirm the Signal
signals_df.loc["2015-02-09":"2015-02-17"]

,close,SMA50,SMA100,Signal
date,,,,
2015-02-09,119.72,112.7846,NaN,0.0
2015-02-10,122.02,112.8450,NaN,0.0
2015-02-11,124.88,112.9640,NaN,0.0
2015-02-12,126.46,113.1918,109.71095,1.0
2015-02-13,127.08,113.4408,109.97115,1.0
2015-02-17,127.83,113.6788,110.22305,1.0


In [39]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.loc["2015-02-09":"2015-02-17"]

,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
2015-02-09,119.72,112.7846,NaN,0.0,0.0
2015-02-10,122.02,112.8450,NaN,0.0,0.0
2015-02-11,124.88,112.9640,NaN,0.0,0.0
2015-02-12,126.46,113.1918,109.71095,1.0,1.0
2015-02-13,127.08,113.4408,109.97115,1.0,0.0
2015-02-17,127.83,113.6788,110.22305,1.0,0.0


In [41]:
# Visualize exit position relative to close price
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["close"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
exit

:Scatter   [date]   (close)

In [42]:
# Visualize entry position relative to close price
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["close"].hvplot.scatter(
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
entry

:Scatter   [date]   (close)

In [43]:
# Visualize close price for the investment
security_close = signals_df[["close"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
security_close

:Curve   [date]   (close)

In [44]:
# Visualize moving averages
moving_avgs = signals_df[["SMA50", "SMA100"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [45]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)